In [55]:
df_nes_learn.head()

,sku,categoryLevel2Id,brandId,userName,reting,comment,commentNegative,commentPositive
0,20005023,4010201,826,b2898a81b45310b30beb8fc0c0a9ce1e,2.0,"2,5 года работала и все...устала! Лампочка гор...",NaN,NaN
1,20020647,4030101,1425,538c73d64461e13907bb95c51c38bfbc,2.0,Через 2 месяца после истечении гарантийного ср...,NaN,NaN
2,20020701,4010401,124,ddca2d0101513a6209db7868eed8be05,4.0,пользуюсь уже три недели. нареканий ни каких н...,NaN,NaN
3,30012256,2030301,93,289c20015b3713a82ba5ddf774d996f7,5.0,Ребят этот системный блок подойдёт для игры кс...,NaN,NaN
4,30011341,2050201,656,5576f82d149d4f688644fef2322c63ef,5.0,"я считаю, что яри замечательный телефон! Прият...",NaN,NaN


In [27]:
import re
import nltk
from nltk.stem.porter import PorterStemmer

def get_tokens(stem):
    tokens = nltk.word_tokenize(stem)
    return tokens

def do_stemming(filtered):
    stemmed = []
    for f in filtered:
        stemmed.append(PorterStemmer().stem(f))  # не используется? снесите нах
    return stemmed

class Porter:
    PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    @staticmethod  # здесь не хватало staticmethod'а
    def stem(word):
        word = word.lower()
        word = word.replace(u'ё', u'е')
        m = re.match(Porter.RVRE, word)

        if m and m.groups():
            pre = m.group(1)
            rv = m.group(2)
            temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
            if temp != rv:
                rv = temp

            rv = Porter.REFLEXIVE.sub('', rv, 1)
            temp = Porter.ADJECTIVE.sub('', rv, 1)
            if temp != rv:
                rv = temp
                rv = Porter.PARTICIPLE.sub('', rv, 1)
            else:
                temp = Porter.VERB.sub('', rv, 1)
                if temp == rv:
                    rv = Porter.NOUN.sub('', rv, 1)
                else:
                    rv = temp

            rv = Porter.I.sub('', rv, 1)
            if re.match(Porter.DERIVATIONAL, rv):
                rv = Porter.DER.sub('', rv, 1)
                temp = Porter.P.sub('', rv, 1)
            if temp == rv:
                rv = Porter.SUPERLATIVE.sub('', rv, 1)
                rv = Porter.NN.sub(u'н', rv, 1)
            else:
                rv = temp
            word = pre+rv
        return word



# импортируем Pandas и Numpy
import re
import pandas as pd
import numpy as np
df = pd.read_csv('X_train.csv')
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 1000)
cols = list(df.columns.values)

df_nes=df[['sku', 'categoryLevel2Id' , 'brandId', 'userName', 'reting' , 'comment' , 'commentNegative', 'commentPositive']]

#df_nes.head(n=100)
df_nes_learn=df_nes.head(n=10000)
df_nes_test=df_nes.tail(n=5550)
#df_nes_test.head()
#df_nes_learn.head()


abs=set()
abs_b=set()


for row in list(df_nes_learn.itertuples(index=True, name='Pandas')):
#for row in df_nes_learn.itertuples(index=True, name='Pandas'):
    tokens = get_tokens(getattr(row,'comment'))
    

    for token in tokens:
        if re.match(r'\w+', token):
            stemmed = Porter.stem(token)
            abs.add(stemmed)
            abs_b.add(token)
            # print(token, '->', stemmed)
    # а можно сразу так:
 
    #stems = [Porter.stem(token) for token in tokens if re.match(r'\w+', token)]
    #print('оригинальный текст:')
    #print(row.comment)
    #print()
    #print('стемы:')
    #print(' '.join(stems))
    #print('\n' + '-' * 60 + '\n')
    # print('                                      ')
print(len(abs))
print(len(abs_b))



34545
53901


In [56]:
df_nes_test.head()

,sku,categoryLevel2Id,brandId,userName,reting,comment,commentNegative,commentPositive
10037,20023434,4050301,64,33b67031842d4c56d9dc03571081b77d,5.0,"Покупкой доволен,прежде чем купить изучал инос...",NaN,NaN
10038,20021798,4110104,10,fc7d8d59ae565bbc0d43d6a24d0f888f,5.0,Пылесос просто супер. очень довольны!,NaN,NaN
10039,20021697,4110102,80,6421219d872c945f8aa9dc434c923394,5.0,Пользуюсь месяц и не вижу ни каких проблем. Мо...,NaN,NaN
10040,11023744,1070908,13,3d801da09e7d82668e226799d9db91dc,2.0,Купил я этот телевизор Маме на день рождение. ...,NaN,NaN
10041,40054798,3030501,1542,560e1a279bae50a0248e626224828b4f,5.0,Лучшего симулятора я ещё не встречал. Хорошая ...,NaN,NaN


In [83]:
import nltk
from nltk.stem.porter import PorterStemmer



def get_tokens(stem):
    tokens = nltk.word_tokenize(stem)
    return tokens

def do_stemming(filtered):
    stemmed = []
    for f in filtered:
        stemmed.append(PorterStemmer().stem(f))
    return stemmed




In [117]:
class Porter:
	PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
	REFLEXIVE = re.compile(u"(с[яь])$")
	ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
	PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
	VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
	NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
	RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
	DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
	DER = re.compile(u"ость?$")
	SUPERLATIVE = re.compile(u"(ейше|ейш)$")
	I = re.compile(u"и$")
	P = re.compile(u"ь$")
	NN = re.compile(u"нн$")

	def stem(word):
		word = word.lower()
		word = word.replace(u'ё', u'е')
		m = re.match(Porter.RVRE, word)
		if m.groups():
			pre = m.group(1)
			rv = m.group(2)
			temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
			if temp == rv:
				rv = Porter.REFLEXIVE.sub('', rv, 1)
				temp = Porter.ADJECTIVE.sub('', rv, 1)
				if temp != rv:
					rv = temp
					rv = Porter.PARTICIPLE.sub('', rv, 1)
				else:
					temp = Porter.VERB.sub('', rv, 1)
					if temp == rv:
						rv = Porter.NOUN.sub('', rv, 1)
					else:
						rv = temp
			else:
				rv = temp
			
			rv = Porter.I.sub('', rv, 1)

			if re.match(Porter.DERIVATIONAL, rv):
				rv = Porter.DER.sub('', rv, 1)

			temp = Porter.P.sub('', rv, 1)
			if temp == rv:
				rv = Porter.SUPERLATIVE.sub('', rv, 1)
				rv = Porter.NN.sub(u'н', rv, 1)
			else:
				rv = temp
			word = pre+rv
		return word

In [26]:
len(abs)

34545